In [238]:
# Import libraries
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import numpy as np
import re
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import warnings
import random
import csv
warnings.filterwarnings("ignore")

In [239]:
# Import csv file
cities_path = "../data/hotels_items.csv"

In [240]:
# Create dataframe
cities_df = pd.read_csv(cities_path, usecols = ["city"])

In [241]:
cities_df.drop_duplicates(inplace = True)

In [242]:
city_list = cities_df["city"].tolist()

In [243]:
# # Create empty columns for the values that will be pulled through API
hotels_df = pd.DataFrame()
hotels_df["rating"] = np.nan
hotels_df["name"] = None
hotels_df["address"] = None
hotels_df["locality"] = None
hotels_df["landmarks"] = None
hotels_df["price"] = np.nan

In [244]:
# List lenght
len(city_list)

27194

In [247]:
options = Options()
options.headless = True
#options.add_argument("--window-size=1024x768")
driver = webdriver.Chrome('/usr/bin/chromedriver', chrome_options=options)
url = "https://www.hotels.com/"

#driver.implicitly_wait(2)

with open(r'../data/hotels_com_scrape_v2.csv', 'a') as f:
    csv_writer = csv.writer(f)
    for city in list(city_list):
        #print('Processing: ' + city)

        searchKey = city
        checkInDate = '08/27/2020' #Format %m/%d/%Y
        checkOutDate = '08/29/2020' #Format %m/%d/%Y

        driver.get(url)

        searchKeyElement = driver.find_elements_by_name("q-destination")
        checkInElement = driver.find_elements_by_name("q-localised-check-in")
        checkOutElement = driver.find_elements_by_name("q-localised-check-out")
        submitButton = driver.find_elements_by_css_selector("button.cta-strong")
        closeIconForPopup = driver.find_elements_by_css_selector("span.close-icon")
        closeTypeahead = driver.find_elements_by_css_selector("button.cta-link")
        closeTypeahead = driver.find_elements_by_css_selector("button.cta-link")

        if searchKeyElement and checkInElement and checkOutElement and closeIconForPopup:
            #print('All search elements found.')
            searchKeyElement[0].clear()
            searchKeyElement[0].send_keys(searchKey)
            checkInElement[0].clear()
            checkInElement[0].send_keys(checkInDate)
            checkOutElement[0].clear()
            checkOutElement[0].send_keys(checkOutDate)
            closeIconForPopup[0].click()
            time.sleep(1)
            
            try:
                submitButton[0].click()
            except:                
                # Try to close the the autocomplete box
                time.sleep(1)
                searchKeyElement[0].send_keys(Keys.ARROW_DOWN)
                time.sleep(1)
                searchKeyElement[0].send_keys(Keys.ENTER)
            
            time.sleep(1)
            #print('Pressed the submit button.')

            #print('Scrolling a few times.')
            #body_element = driver.find_element_by_tag_name('body')
            lastElement = driver.find_elements_by_tag_name("div")[-1]
            for i in range(2):
                lastElement.send_keys(Keys.NULL)
            
            time.sleep(1)
            #print(driver.page_source)
            hotels = driver.find_elements_by_css_selector('section.hotel-wrap')

            for index, hotel in enumerate(hotels):
                hotel_name = hotel.find_element_by_css_selector('a.property-name-link').text.strip()
                hotel_url = hotel.find_element_by_css_selector('a.property-name-link').get_attribute("href")
                try:
                    hotel_rating = float(hotel.find_element_by_css_selector('div.ta-logo').text.split()[-1])
                except:
                    hotel_rating = "na"
                hotel_address = hotel.find_element_by_css_selector('span.address').text
                hotel_locality = hotel.find_element_by_css_selector("a.map-link").text.strip()
                hotel_landmarks = hotel.find_element_by_css_selector("ul.property-landmarks").text.replace("\n", ", ")
                try:
                    hotel_price = float(hotel.find_element_by_css_selector("a.price-link").find_element_by_tag_name("ins").text.strip("$"))
                except:
                    hotel_price = "na"

                #print("Saving...")
                row = [city, hotel_name, hotel_rating, hotel_address, hotel_locality, hotel_landmarks, hotel_price, hotel_url]
                print(row)
                csv_writer.writerow(row)
        #print("Done!")
        city_list.remove(city)
        print("\n" + str(len(city_list)) + " cities remaining. \n")

driver.quit()
print("We done.")

ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=75.0.3770.80)


In [ ]:
driver.quit()